In [54]:
import pandas as pd 
import numpy as np 
import xgboost
import os
import time
import warnings
import sklearn
import seaborn as sns 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt 
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')

In [55]:
file=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/matrix_0.csv'
weatherfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/weather_data_2014-1-1~2016~9-30.xlsx'
numst='1416'

In [56]:
data=pd.read_csv(file)
weatherdata=pd.read_excel(weatherfile)
data=data.set_index('Unnamed: 0')
data.columns=[str(x) for x in range(1,1455)]
weatherdata=weatherdata.set_index('date')

def tz(df,str1):
    df['timestamp']=df.index
    df['time_stamp']=pd.DataFrame(pd.to_datetime(df['timestamp']))
    df['dow'] = df.time_stamp.apply(lambda x: x.dayofweek)
    df['doy'] = df.time_stamp.apply(lambda x: x.dayofyear)
    df['day'] = df.time_stamp.apply(lambda x: x.day)
    df['month'] = df.time_stamp.apply(lambda x: x.month)
    df['d-1']=df[str1].shift(1)
    df['d-2']=df[str1].shift(2)
    df['d_dif']=df['d-1']-df['d-2']
    weatherdata['date']=weatherdata.index
    combineddf=pd.merge(left=df,right=weatherdata,left_on='time_stamp',right_on='date',how='left')
    return combineddf

combineddf=tz(pd.DataFrame(data[numst]),numst)
combineddf=combineddf.set_index('time_stamp')

X=combineddf[['dow','doy','day','month','d-1','d-2','d_dif','max','min','max-min']]
Y=combineddf[[numst]]

Xtrain=X[X.index<'2016-07-01']
Ytrain=Y[Y.index<'2016-07-01']

Xtest=X[(X.index>='2016-07-01')&(X.index<'2016-08-01')]
Ytest=Y[(Y.index>='2016-07-01')&(Y.index<'2016-08-01')]

#rfr=xgboost.sklearn.XGBRegressor()
rfr=GradientBoostingRegressor(max_depth=6,n_estimators=500,learning_rate=0.1,random_state=1)

In [57]:
rfr.fit(Xtrain,Ytrain)
print(rfr.score(Xtrain,Ytrain))
print(rfr.score(Xtest,Ytest))

result=pd.read_excel(r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/result.xlsx')

result['dow'] = result.date.apply(lambda x: x.dayofweek)
result['doy'] = result.date.apply(lambda x: x.dayofyear)
result['day'] = result.date.apply(lambda x: x.day)
result['month'] = result.date.apply(lambda x: x.month)

combinedresultdf=pd.merge(left=result,right=weatherdata,left_on='date',right_on='date',how='left')

combinedresultdf=combinedresultdf[['date','dow','doy','day','month','max','min','max-min']]

combinedresultdf['d-1']=0
combinedresultdf['d-2']=0
combinedresultdf['d_dif']=0

combinedresultdf=combinedresultdf.set_index('date')

combinedresultdf.head()
c2=combinedresultdf



days=list(c2.index)

c2=c2.reset_index()

# c2.loc[2,'d-1']=Ytest[Ytest.index=='2016-08-31'].iloc[0,0]

# c2.loc[2,'d-2']=Ytest[Ytest.index=='2016-08-30'].iloc[0,0]

c2.loc[1,'d_dif']=c2.loc[1,'d-1']-c2.loc[1,'d-2']


namelist=['dow','doy','day','month','d-1','d-2','d_dif','max','min','max-min']
c2.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [45]:
c2.loc[0,'d-1']=Y.loc['2016-08-31'][0]
c2.loc[0,'d-2']=Y.loc['2016-08-30'][0]
c2.loc[0,'d_dif']=c2.loc[0,'d-1']-c2.loc[0,'d-2']
c2.loc[1,'d-2']=Y.loc['2016-08-31'][0]

c2.head()

,date,dow,doy,day,month,max,min,max-min,d-1,d-2,d_dif
0,2016-09-01,3,245,1,9,33,23,10,563819,552407,11412
1,2016-09-02,4,246,2,9,34,22,12,0,563819,0
2,2016-09-03,5,247,3,9,34,23,11,0,0,0
3,2016-09-04,6,248,4,9,31,22,9,0,0,0
4,2016-09-05,0,249,5,9,32,23,9,0,0,0


In [46]:
for x in range(30):
    if x!=29:
        a=rfr.predict(pd.DataFrame(c2.loc[x,namelist]).T)[0]
        c2.loc[x+1,'d-1']=a
        c2.loc[x+2,'d-2']=a
        c2.loc[x,'d_dif']=c2.loc[x,'d-1']-c2.loc[x,'d-2']
        c2.loc[x,'pre']=a
    elif x==29:
        a=rfr.predict(pd.DataFrame(c2.loc[x,namelist]).T)[0]
        c2.loc[x,'pre']=a
        c2.loc[x,'d_dif']=c2.loc[x,'d-1']-c2.loc[x,'d-2']

c2=c2[c2.index<30]

In [10]:
filename=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/'+numst+r'.xlsx'
filename

c2[['date','pre']].to_excel(filename)

In [53]:
combineddf[combineddf.index==r'2016-07-05']

,1416,timestamp,dow,doy,day,month,d-1,d-2,d_dif,max,min,weather,wind,wind_speed,max-min,date
time_stamp,,,,,,,,,,,,,,,,


In [40]:
Xtest

,dow,doy,day,month,d-1,d-2,d_dif,max,min,max-min
time_stamp,,,,,,,,,,
2016-07-01,4,183,1,7,13830.0,13829.0,1.0,26.0,23.0,3.0
2016-07-02,5,184,2,7,14678.0,13830.0,848.0,26.0,22.0,4.0
2016-07-03,6,185,3,7,13983.0,14678.0,-695.0,25.0,22.0,3.0
2016-07-04,0,186,4,7,12992.0,13983.0,-991.0,27.0,22.0,5.0
2016-07-12,1,194,12,7,14743.0,12792.0,1951.0,32.0,26.0,6.0
2016-07-13,2,195,13,7,14314.0,14743.0,-429.0,32.0,26.0,6.0
2016-07-14,3,196,14,7,14984.0,14314.0,670.0,29.0,26.0,3.0
2016-07-15,4,197,15,7,14687.0,14984.0,-297.0,33.0,22.0,11.0
2016-07-16,5,198,16,7,14421.0,14687.0,-266.0,29.0,22.0,7.0


In [38]:
Ytest.shape

(24, 1)

In [51]:
data[data.index==r'2016/7/6']

,1,2,3,4,5,6,7,8,9,10,...,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2016/7/6,460,209,443,816,895,364,79,839,398,337,...,334,169,454,113,333,794,2821,70,1366,57
